In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import tensorflow as tf
from keras.metrics import Accuracy
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
import os
import cv2
from PIL import Image
import numpy as np
from tensorflow.keras import Model

2024-07-24 04:32:54.610622: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 04:32:54.610764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 04:32:54.786948: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
image_directory = '/kaggle/input/dataset/train'
SIZE = 224
X_train = []
y_train = []

cancerous = os.listdir(image_directory + '/Normal/')
for i, image_name in enumerate(cancerous):    #Remember enumerate method adds a counter and returns the enumerate object


        image = cv2.imread(image_directory + '/Normal/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        X_train.append(np.array(image))
        y_train.append(1)

non_cancerous = os.listdir(image_directory + '/OSCC/')
for i, image_name in enumerate(non_cancerous):    #Remember enumerate method adds a counter and returns the enumerate object


        image = cv2.imread(image_directory + '/OSCC/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        X_train.append(np.array(image))
        y_train.append(0)

print(len(X_train))
print(len(y_train))

4946
4946


In [2]:
image_directory = '/kaggle/input/dataset/val'
SIZE = 224
X_val = []
y_val = []

cancerous = os.listdir(image_directory + '/Normal/')
for i, image_name in enumerate(cancerous):    #Remember enumerate method adds a counter and returns the enumerate object


        image = cv2.imread(image_directory + '/Normal/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        X_val.append(np.array(image))
        y_val.append(1)

non_cancerous = os.listdir(image_directory + '/OSCC/')
for i, image_name in enumerate(non_cancerous):    #Remember enumerate method adds a counter and returns the enumerate object


        image = cv2.imread(image_directory + '/OSCC/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        X_val.append(np.array(image))
        y_val.append(0)

print(len(X_val))
print(len(y_val))

120
120


In [4]:
image_directory = '/kaggle/input/dataset/test'
SIZE = 224
X_test = []
y_test = []

cancerous = os.listdir(image_directory + '/Normal/')
for i, image_name in enumerate(cancerous):    #Remember enumerate method adds a counter and returns the enumerate object


        image = cv2.imread(image_directory + '/Normal/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        X_test.append(np.array(image))
        y_test.append(1)

non_cancerous = os.listdir(image_directory + '/OSCC/')
for i, image_name in enumerate(non_cancerous):    #Remember enumerate method adds a counter and returns the enumerate object


        image = cv2.imread(image_directory + '/OSCC/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        X_test.append(np.array(image))
        y_test.append(0)

print(len(X_test))
print(len(y_test))

126
126


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.utils import to_categorical

# Define the self-attention layer
def self_attention(input_tensor):
    attention = layers.Conv2D(filters=input_tensor.shape[-1], kernel_size=1, padding='same', activation='sigmoid')(input_tensor)
    output_tensor = layers.Multiply()([input_tensor, attention])
    return output_tensor

# Define the spatial attention layer as a custom layer
class SpatialAttention(layers.Layer):
    def __init__(self, **kwargs):
        super(SpatialAttention, self).__init__(**kwargs)
        self.conv = layers.Conv2D(filters=1, kernel_size=7, padding='same', activation='sigmoid')
    
    def call(self, inputs):
        avg_pool = tf.reduce_mean(inputs, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(inputs, axis=-1, keepdims=True)
        concat = tf.concat([avg_pool, max_pool], axis=-1)
        attention = self.conv(concat)
        output = inputs * attention
        return output

# Define the soft attention layer as a custom layer
class SoftAttention(layers.Layer):
    def __init__(self, **kwargs):
        super(SoftAttention, self).__init__(**kwargs)
        self.dense = layers.Dense(units=None, activation='sigmoid')
    
    def build(self, input_shape):
        self.dense.units = input_shape[-1]
    
    def call(self, inputs):
        attention = layers.GlobalAveragePooling2D()(inputs)
        attention = self.dense(attention)
        attention = tf.reshape(attention, [-1, 1, 1, inputs.shape[-1]])
        output = inputs * attention
        return output

# Create the model
def create_model(input_shape=(224, 224, 3), num_classes=2):
    base_model = EfficientNetB0(include_top=False, input_shape=input_shape, weights='imagenet')
    base_model.trainable = True  # Freeze the base model

    inputs = layers.Input(shape=input_shape)
    x = base_model(inputs, training=True)

    # Apply Self Attention
    x = self_attention(x)
    x = SpatialAttention()(x)
    x = SoftAttention()(x)
    x = Dense(1024,activation='relu')(x)
    x = Dense(512,activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = Dense(512,activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(2, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

# Instantiate and compile the model
model = create_model()

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * tf.math.exp(-0.1).numpy())

lr_scheduler = LearningRateScheduler(scheduler)

# Model checkpoint
model_checkpoint = ModelCheckpoint(
    'best_model_updated-1.keras', 
    monitor='val_accuracy', 
    save_best_only=True, 
    mode='max'
)

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001,
    patience=30,
    verbose=1,
    mode='max',
    restore_best_weights=True,
    start_from_epoch=10
)

# Assuming you have X_train, X_test, y_train, y_test
# Fit the model
history = model.fit(np.array(X_train),
                    to_categorical(np.array(y_train)),
                    batch_size=4,
                    validation_data=(np.array(X_val), to_categorical(np.array(y_val))),
                    epochs=60,
                    callbacks=[lr_scheduler, model_checkpoint,early_stopping]
                    )

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Assuming you have your true labels (y_true) and predicted labels (y_pred)
y_true = y_test
y_pred = model.predict(np.array(X_test))
y_pred = np.argmax(y_pred, axis=1)
target_names = ["OSCC", "Normal"]

# Generate classification report
report_str = classification_report(y_true, y_pred, target_names=target_names, zero_division=1)
print(report_str)  # Print the classification report string

# Generate confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=[0, 1])

# Display the confusion matrix using ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_names)
disp.plot(cmap='Blues')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix')
plt.show()